In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
def base_no_horizontal(base):
    base = base[
        (base['porteria'].isna()) &
        (base['conjuntocerrado']==0) &
        (base['tipoinmueble']=='Casa') &
        (base['ascensor'].isna()) &
        ((base['gimmasio'].isna()) | (base['gimnasio']==0)) &
        (base['piscina'].isna())
        #Si encontramos gimnasios en la geocodificación comparar con los puntos que se quitaron
    ]
    return base

In [ ]:
carpeta_path = r'C:\Users\William\Desktop\el algoritmo es correcto'
archivo_train = r'train_data.csv'
path_train = os.path.join(carpeta_path, archivo_train)

In [ ]:
path_train

In [ ]:
train = pd.read_csv(
    path_train, 
    dtype={
        "tiempodeconstruido": str,
        "vista": str,
        "tipoinmueble": str,
        "tiponegocio": str,
        'valorventa': 'int64',
        # 'area': 'int64'
    }
)

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
for columna in train.columns:
    if columna not in ['id', 'valoradministracion', 'valorventa', 'latitud', 'longitud']:
        print("{}: ".format(columna), train[columna].unique())

In [ ]:
prueba = train[
    (train['porteria'].isna()) &
    (train['conjuntocerrado']==0) &
    (train['tipoinmueble']=='Casa') &
    (train['ascensor'].isna()) &
    ((train['gimnasio'].isna()) | (train['gimnasio']==0)) &
    (train['piscina'].isna())
]#.groupby(['estrato','banos'], as_index=False).agg(**{'Cantidad': pd.NamedAgg('area', 'count')}).head(30)

## Analizar columnas

### area

* file:///C:/Users/William/Downloads/406-Article%20Text-891-1-10-20181207%20(1).pdf

In [ ]:
train['area'].describe().astype('int64')

In [ ]:
train['area'].quantile(np.linspace(0,1,21)).astype('int64')

In [ ]:
train[train['area']<=100]['area'].plot(kind='box')

In [ ]:
train[
    (train['area']>=500) &
    (train['area']<=10000)]['area'].plot(kind='box')

In [ ]:
train[(train['area']>=1000)][['habitaciones', 'banos', 'estrato']].plot(kind = 'box')

In [ ]:
train[(train['area']>=500) & (train['estrato']==0)]

In [ ]:
# Propiedades con más de 20 años  es decir los que tengan más de 60 mts2
train[
    (train['tiempodeconstruido'].isin(['Entre 10 y 20 años','Más de 20 años','16 a 30 años'])) &
    (train['area']<60)
].shape

In [ ]:
# Propiedades con más de 30 años  es decir los que tengan más de 80 mts2
train[
    (train['tiempodeconstruido']=='Más de 30 años') &
    (train['area']<80)
].shape

In [ ]:
# Propiedades con menos de 20 años  es decir los que tengan más de 35 mts2
train[
    (train['tiempodeconstruido'].isin(
        ['Entre 10 y 20 años', 'Entre 0 y 5 años', 'Entre 5 y 10 años',
        '1 a 8 años', '9 a 15 años','Menos de 1 año','ntre 0 y 5 años'])) &
    (train['area']<35)
].shape

In [ ]:
# Propiedades que de entrada no cumplen los 35 mts
train[(train['area']<35)].shape

In [ ]:
train[(train['area']<35)][['id','area', 'habitaciones', 'banos', 'valorventa']]

In [ ]:
train.loc[10]

In [ ]:
train["y"] = train["valorventa"]/train["area"]

In [ ]:
# train["y"]
f = train[
    (train.valorventa < 800000000) & 
    (train.valorventa > 100000000) &
    (train.area < 5000) &
    (train.area > 35) &
    (train.habitaciones.notna()) &
    (train.banos.notna()) &
    (train.estrato.isin([4,6,7])) &
    (train.habitaciones.isin([1,3,8]))
]
f[["y","estrato", "habitaciones"]].boxplot(by=["estrato", "habitaciones"], figsize=(10,10))

In [ ]:
# id: 721798, 759440 Con estrato 0 son haciendas
# Zonas retiradas de bogotá

In [ ]:
import geopandas

In [ ]:
localidades_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\localidades.json'
localidades = geopandas.read_file(localidades_path, crs={'init':'epsg:4686'})

In [ ]:
localidades_geo = localidades.plot(figsize=(10, 10), column='LocCodigo', zorder=1)

In [ ]:
estratificacion_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\manzana_estratificacion.json'
estratificacion = geopandas.read_file(estratificacion_path, crs={'init':'epsg:4686'})

In [ ]:
casas = geopandas.GeoDataFrame(
    prueba, geometry=geopandas.points_from_xy(prueba.longitud, prueba.latitud), crs={'init': 'epsg:4326'})

In [ ]:
localidades_geo = localidades.plot(color='white', edgecolor='black', figsize=(10, 10), zorder=1)
estratificacion_geo = estratificacion.plot(ax=localidades_geo, figsize=(10, 10), zorder=2, cmap='OrRd', legend=True)
casas_geo = casas.plot(ax=localidades_geo, figsize=(10, 10), markersize=.1, zorder=3, marker='x')

In [ ]:
estrato_intersectado = geopandas.sjoin(left_df=casas, right_df=estratificacion, how='left', op='intersects')

In [ ]:
estrato_intersectado['ESTRATO'] = np.where(
    estrato_intersectado['ESTRATO'].isna() | estrato_intersectado['ESTRATO']==0,
    np.NaN,
    estrato_intersectado['ESTRATO']
)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
train = estrato_intersectado.loc[estrato_intersectado['ESTRATO'].notna(),['ESTRATO', 'latitud', 'longitud']]
# test = estrato_intersectado.loc[estrato_intersectado['ESTRATO'].isnull(),['ESTRATO', 'latitud', 'longitud']]

In [ ]:
X = train[['latitud', 'longitud']].values
y = train[['ESTRATO']].values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Se crea el modelo con la base de entrenamiento
resultados = {}
for i in range(1,12):
    neigh = KNeighborsClassifier(n_neighbors=i, metric = 'euclidean')
    neigh.fit(X_train, y_train.ravel())
    resultados[i] = neigh.score(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(list(resultados.keys()), list(resultados.values()))
plt.show()

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3, metric = 'euclidean')
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)

In [ ]:
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))

In [ ]:
valores = estrato_intersectado.loc[estrato_intersectado['ESTRATO'].isnull(),['latitud', 'longitud']].values

In [ ]:
estrato_intersectado.loc[estrato_intersectado['ESTRATO'].isnull(),'ESTRATO'] = neigh.predict(valores)

In [ ]:
estrato_intersectado.groupby(['ESTRATO'], as_index=False).agg(**{'Cantidad': pd.NamedAgg('latitud','count')})

In [ ]:
a = estrato_intersectado['estrato'] - estrato_intersectado['ESTRATO']
pd.DataFrame(a, columns=['Dif Estrato']).reset_index(drop=False).rename(columns={'index':'Cantidad'}).groupby('Dif Estrato', as_index=False).count()